In [ ]:
# Testing differenyt interpolation methods

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
import napari

In [3]:
viewer = napari.Viewer()
viewer.open_sample('napari', 'cells3d')
membrane_layer = viewer.layers[0]
nuclei_layer = viewer.layers[1]

Assistant skips harvesting pyclesperanto as it's not installed.


C:\Users\james\anaconda3\envs\napari\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [4]:
from scipy.ndimage import distance_transform_edt
from scipy.interpolate import interpn

def distance_transform(image):
    """Distance transform for a boolean image.
    
    Returns positive values inside the object,
    and negative values outside.
    """
    image = image.astype(bool)
    edt = distance_transform_edt(image) - distance_transform_edt(~image)
    return edt


def point_and_values(image_1, image_2, interp_dim=0):
    edt_1 = distance_transform(image_1)
    edt_2 = distance_transform(image_2)
    values = np.stack([edt_1, edt_2], axis=interp_dim)
    points = tuple([np.arange(i) for i in values.shape])
    return points, values


def xi_coords(shape, percent=0.5, interp_dim=0):
    slices = [slice(0, i) for i in shape]
    xi = np.moveaxis(np.mgrid[slices], 0, -1).reshape(np.prod(shape), len(shape)).astype('float')
    xi = np.insert(xi, interp_dim, percent, axis=1)
    return xi


def slice_iterator(slice_index_1, slice_index_2):
    intermediate_slices = np.arange(slice_index_1 + 1, slice_index_2)
    n_slices = slice_index_2 - slice_index_1 + 1  # inclusive
    stepsize = 1 / n_slices
    intermediate_percentages = np.arange(0 + stepsize, 1, stepsize)
    return zip(intermediate_slices, intermediate_percentages)


def interpolated_slice(percent, points, values, interp_dim=0, method='linear'):
    # Find the original image shape
    img_shape = list(values.shape)
    del img_shape[interp_dim]
    # Calculate the interpolated slice
    xi = xi_coords(img_shape, percent=percent, interp_dim=interp_dim)
    interpolated_img = interpn(points, values, xi, method=method)
    interpolated_img = np.reshape(interpolated_img, img_shape) > 0
    return interpolated_img
    

In [5]:
def interpolate_between_slices(label_layer: "napari.layers.Labels", slice_index_1: int, slice_index_2: int, label_id: int =1, interp_dim: int =0):
    if slice_index_1 > slice_index_2:
        slice_index_1, slice_index_2 = slice_index_2, slice_index_1
    layer_data = np.asarray(label_layer.data)
    slice_1 = np.take(layer_data, slice_index_1, axis=interp_dim)
    slice_2 = np.take(layer_data, slice_index_2, axis=interp_dim)
    # slice_1 = np.asarray(label_layer.data[slice_index_1])
    # slice_2 = np.asarray(label_layer.data[slice_index_2])

    #TODO: possible extension, handle all label ids separately     
    slice_1 = slice_1.astype(bool)
    slice_2 = slice_2.astype(bool)
    # interp_dim should just be the slider "dimension" right?
    points, values = point_and_values(slice_1, slice_2, interp_dim)
    #TODO: Thread this?   
    for slice_number, percentage in slice_iterator(slice_index_1, slice_index_2):
        interpolated_img = interpolated_slice(percentage, points, values, interp_dim=interp_dim, method='linear')
        indices = [slice(None) for _ in range(label_layer.data.ndim)]
        indices[interp_dim] = slice_number
        indices = tuple(indices)
        label_layer.data[indices][interpolated_img] = label_id
    label_layer.refresh()  # will update the current view

In [22]:
label_layer = viewer.layers[-1]
slice_index_1 = 24
slice_index_2 = 30
slice_index_3 = 35
slice_index_4 = 40
slice_index_5 = 45
slices_indices = [24, 30, 35, 40, 45]
#get layers at these slices
# add the an empty 3d volume (or same size) (np.zeroes)
from scipy.ndimage import map_coordinates
interp_dim = 0
empty_cube = np.zeros(shape=(60, 256, 256), dtype="uint8")

slice_1 = np.take(label_layer.data, slice_index_1, axis=interp_dim)
slice_2 = np.take(label_layer.data, slice_index_2, axis=interp_dim)
slice_3 = np.take(label_layer.data, slice_index_3, axis=interp_dim)
slice_4 = np.take(label_layer.data, slice_index_4, axis=interp_dim)
slice_5 = np.take(label_layer.data, slice_index_5, axis=interp_dim)

empty_cube[slice_index_1] = slice_1
empty_cube[slice_index_2] = slice_2
empty_cube[slice_index_3] = slice_3
empty_cube[slice_index_4] = slice_4
empty_cube[slice_index_5] = slice_5

# import tifffile as t
# t.imwrite("label_fiveslices", empty_cube)




In [25]:
# interpolate_between_slices(label_layer, slice_index_1, slice_index_2, interp_dim=0)

In [14]:
dir(label_layer.data)

['T',
 '_Oindex',
 '_Vindex',
 '__array__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_unpickle',
 'astype',
 'chunk_layout',
 'codec',
 'dimension_units',
 'domain',
 'dtype',
 'fill_value',
 'kvstore',
 'mode',
 'ndim',
 'oindex',
 'origin',
 'rank',
 'read',
 'readable',
 'resolve',
 'schema',
 'shape',
 'size',
 'spec',
 'transaction',
 'vindex',
 'with_transaction',
 'writable',
 'write']